# Data Analysis with Decision Tree and Random Forest model for the Bank Personal Loan Case study. Target Variable is Personal loan.We have to analyse what are the customers can be predicted as accepting the personal loan based on various input features.

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

C:\Users\rajes\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [20]:
ds=pd.read_excel('C:/Users/rajes/Desktop/LetsUpgrade/AI-ML July20/Day 22-24/Bank_Personal_Loan_Modelling.xlsx',sheet_name=1)

In [21]:
ds.head(2)

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0


In [22]:
ds.columns

Index(['ID', 'Age', 'Experience', 'Income', 'ZIP Code', 'Family', 'CCAvg',
       'Education', 'Mortgage', 'Personal Loan', 'Securities Account',
       'CD Account', 'Online', 'CreditCard'],
      dtype='object')

In [23]:
# Since ID and ZIP code does not have much significance , so dropping them from feature set.
ds1=ds.drop(['ID','ZIP Code'],axis=1)

In [24]:
ds1.head(2)

,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,25,1,49,4,1.6,1,0,0,1,0,0,0
1,45,19,34,3,1.5,1,0,0,1,0,0,0


In [26]:
# Removing Null values if any
ds2=ds1.dropna()
#Removing duplicate values
ds3=ds2.drop_duplicates()

In [27]:
# Using numpy round(), converting float values of CCAvg as int
ds3['CCAvg']=np.round(ds3['CCAvg'])

C:\Users\rajes\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [28]:
ds3.head(2)

,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,25,1,49,4,2.0,1,0,0,1,0,0,0
1,45,19,34,3,2.0,1,0,0,1,0,0,0


In [29]:
# initiazing the random forest Classifier
rf_model=RandomForestClassifier(n_estimators=1000,max_features=2,oob_score=True)

In [31]:
# taking input feature set
features=ds3.drop(['Personal Loan'],axis=1)
features.head(1)

,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Securities Account,CD Account,Online,CreditCard
0,25,1,49,4,2.0,1,0,1,0,0,0


In [32]:
rf_model.fit(features,ds3['Personal Loan'])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=2,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=True, random_state=None,
                       verbose=0, warm_start=False)

In [33]:
# Print value of OOB Score ie. Out of Bag score ie Accuracy score of model classification.
print('OOB Score :::')
print(rf_model.oob_score_)

OOB Score :::
0.9865650691798676


In [35]:
# Using RandomForest Algorithm finding the important features that is required for decision making or analysis

for feature, imp in zip(features,rf_model.feature_importances_):
    print(feature,imp)

Age 0.05202129254480128
Experience 0.05121473021547273
Income 0.36160289714865146
Family 0.09886007302739751
CCAvg 0.13588682284503575
Education 0.1703133500477327
Mortgage 0.04802691253977105
Securities Account 0.006282878701426746
CD Account 0.05485634029429027
Online 0.009486466745417524
CreditCard 0.011448235890002881


In [ ]:
# We found that there are 3 variables that has maximum importance i.e higher values as compared to others. SO will will take them in making decision tree.

In [39]:
# Creating Decision Tree model now::
from sklearn import tree
tree_model=tree.DecisionTreeClassifier(max_depth=6,max_leaf_nodes=10)
# Creating Predictor now using 3 variables we finalized through RF
predictors=pd.DataFrame([ds3['Education'],ds3['CCAvg'],ds3['Income']]).T
predictors.head(2)

,Education,CCAvg,Income
0,1.0,2.0,49.0
1,1.0,2.0,34.0


In [40]:
# Fitting to tree model
tree_model.fit(predictors,ds3['Personal Loan'])

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=6, max_features=None, max_leaf_nodes=10,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [67]:
# Creating a decision tree dot file
with open('Dtree.dot','w') as f:
    f=tree.export_graphviz(tree_model,feature_names=['Education','CCAvg','Income'], out_file=f)
    

In [ ]:
# We analyze the created tree file in WebgraphicViz editor.
# Rules that can be extracted from this decision tree:
# Rule 1.  Applicants having Income <=101.5 have very less probability to get the Personal loan.(We decide this from the value on leaf node)
# Rule 2.  IF Applicates have CCAvg <= 3.5 and Income <=88.5 , there is very less chance to get personal loan.
# Rule 3. If Applicants have Education level<=1.5 , there is less chance to get the personal loan.
# Rule 4. If Applicants have Education level>1.5 and Income >=116.5 , there is high probablity of getting Personal loan.
# Rule 5: IF Applicants have Education level<=2.5,  CCAvg>3.5 and Income>88.5, there is high chance to get the personal loan.

In [10]:
ds.head(2)

,Unnamed: 0,Unnamed: 1,Unnamed: 2
0,NaN,NaN,NaN
1,NaN,NaN,NaN
